**Consulta WebService Servicio Metereológico Nacional**

URL: https://smn.conagua.gob.mx/es/web-service-api

- Nombre de servicio: "PronosticoPorMunicipiosGZ" 
    GET tools/GUI/webservices/?method=1
    Descripción: Este 'Web Service' (Servicio Web) ofrece la posibilidad de consumir los datos del pronóstico por municipios por día y a 3 días, formato json (descomprimido), información actualizada cada hora 15 minutos.

- Nombre de servicio: "PronosticoPorMunicipios48HrsGZ"
    GET tools/GUI/webservices/?method=3
    Descripción: Este 'Web Service' (Servicio Web) ofrece la posibilidad de consumir los datos del pronóstico por municipios por hora y a 48 horas, formato json(al descomprimir), información actualizada cada hora 15 minutos.

In [12]:
# Librerías requeridas
import requests
import gzip
import io
import json
import pandas as pd


class ServicioMeteorologicoNacional:
    def __init__(self):
        self.base_url = "https://smn.conagua.gob.mx/tools/GUI/webservices/index.php"
        self.client = requests.Session()
    
    def obtener_informacion(self):
        url = f"{self.base_url}?method=3"
        try:
            # Realizar la solicitud GET
            response = self.client.get(url, stream=True, verify=True)  # Verificación SSL habilitada
            print(f"Status Code: {response.status_code}")
            print(f"Response Headers: {response.headers}")

            if response.status_code == 200:
                # Descomprimir el contenido
                with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz_file:
                    data = gz_file.read()
                
                # Decodificar el contenido JSON
                data_str = data.decode('utf-8')
                print(f"Decoded Content: {data_str[:200]}")  # Print the first 200 characters

                try:
                    data_json = json.loads(data_str)
                    return data_json
                except json.JSONDecodeError as e:
                    print(f"JSON Decode Error: {e}")
            else:
                print(f"Error: Status code {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"HTTP Request Error: {e}")

    def json_a_dataframe(self, json_data):
        # Verificar que json_data es una lista de diccionarios
        if isinstance(json_data, list) and all(isinstance(item, dict) for item in json_data):
            df = pd.DataFrame(json_data)
            return df
        else:
            print("El JSON no está en el formato esperado.")
            return None

# Uso del servicio
servicio = ServicioMeteorologicoNacional()
informacion = servicio.obtener_informacion()

if informacion:
    # Convertir el JSON a DataFrame
    df = servicio.json_a_dataframe(informacion)
    if df is not None:
        print(df.head())  # Muestra las primeras filas del DataFrame
else:
    print("No se pudo obtener la información.")

Status Code: 200
Response Headers: {'Date': 'Thu, 19 Sep 2024 19:25:07 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/8.0.30', 'Content-Description': 'File Transfer', 'Content-Disposition': 'attachment; filename=HourlyForecast_MX.gz', 'Content-Transfer-Encoding': 'binary', 'Expires': '0', 'Cache-Control': 'must-revalidate', 'Pragma': 'public', 'X-Frame-Options': 'SAMEORIGIN, SAMEORIGIN', 'Last-Modified': 'Thu, 19 Sep 2024 19:25:07 GMT', 'Content-Length': '7603078', 'Set-Cookie': 'HttpOnly;Secure, visid_incap_2707069=DyFxpgv4TDeeX6+6TcCQK5J67GYAAAAAQUIPAAAAAABzE6FMTavtgL8+SFysGSQj; expires=Fri, 19 Sep 2025 06:32:56 GMT; HttpOnly; path=/; Domain=.conagua.gob.mx, nlbi_2707069=OR8YEA0WzlPSbnqy51/wewAAAAANWbhit4lKUsq1WkCRum27; HttpOnly; path=/; Domain=.conagua.gob.mx, incap_ses_1808_2707069=TIxTd2u/dUYMSDWnlU4XGZN67GYAAAAAjhp8qjGAIpUVnHne2v8KdA==; path=/; Domain=.conagua.gob.mx', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Content-Security-Policy': "allow 's

In [13]:
df

,desciel,dh,dirvienc,dirvieng,dpt,dsem,hloc,hr,ides,idmun,lat,lon,nes,nhor,nmun,prec,probprec,raf,temp,velvien
0,Despejado,6,Este,107.1,15.4,Miércoles,20240918T18,81.9,20,54,17.3884,-97.229,Oaxaca,-1,Magdalena Zahuatlán,0.0,0.0,9.4,18.5,4.9
1,Cielo nublado,6,Este,91.8,14.6,Miércoles,20240918T19,87.5,20,54,17.3884,-97.229,Oaxaca,0,Magdalena Zahuatlán,0.8,0.0,6.5,16.7,3.9
2,Cielo nublado,6,Noreste,59.1,14.5,Miércoles,20240918T20,90.8,20,54,17.3884,-97.229,Oaxaca,1,Magdalena Zahuatlán,1.2,100.0,5.4,15.9,3.1
3,Cielo nublado,6,Noreste,47.5,14.2,Miércoles,20240918T21,91.2,20,54,17.3884,-97.229,Oaxaca,2,Magdalena Zahuatlán,1.4,100.0,4.8,15.7,3.0
4,Cielo nublado,6,Noreste,60.2,14.0,Miércoles,20240918T22,92.6,20,54,17.3884,-97.229,Oaxaca,3,Magdalena Zahuatlán,1.2,100.0,5.8,15.2,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285703,Poco nuboso,6,Este,80.1,20.7,Lunes,20240923T09,79.0,19,46,25.7553,-100.2896,Nuevo León,110,San Nicolás de los Garza,0.0,0.0,7.2,24.7,4.9
285704,Medio nublado,6,Este,95.6,20.5,Lunes,20240923T10,70.8,19,46,25.7553,-100.2896,Nuevo León,111,San Nicolás de los Garza,0.1,0.0,10.5,26.2,6.7
285705,Poco nuboso,6,Este,97.1,20.1,Lunes,20240923T11,62.8,19,46,25.7553,-100.2896,Nuevo León,112,San Nicolás de los Garza,0.1,0.0,12.2,27.8,8.9
285706,Poco nuboso,6,Este,100.7,19.7,Lunes,20240923T12,56.5,19,46,25.7553,-100.2896,Nuevo León,113,San Nicolás de los Garza,0.1,0.0,13.3,29.2,11.0


In [14]:
# Mapea los nombres actuales a los nuevos nombres
columnas_nuevas = {
    "ides": "id estado",
    "idmun": "id municipio",
    "nes": "nombre estado",
    "nmun": "nombre municipio",
    "dloc": "día local, inicia cuatro horas antes (YYYmmddhhmm)",
    "ndia": "número de día",
    "tmax": "Temperatura máxima (°C)",
    "tmin": "Temperatura mínima (°C)",
    "desciel": "Descripción del cielo",
    "probprec": "Probabilidad de precipitación (%)",
    "prec": "Precipitación (litros/m2)",
    "velvien": "Velocidad del viento (km/h)",
    "dirvienc": "Dirección del viento (Cardinal)",
    "dirvieng": "Dirección del viento (Grados)",
    "cc": "Cobertura de nubes (%)",
    "lat": "Latitud",
    "lon": "Longitud",
    "dh": "Diferencia respecto a hora UTC"
}

# Renombrar las columnas en el DataFrame
df.rename(columns=columnas_nuevas, inplace=True)

In [15]:
df

,Descripción del cielo,Diferencia respecto a hora UTC,Dirección del viento (Cardinal),Dirección del viento (Grados),dpt,dsem,hloc,hr,id estado,id municipio,Latitud,Longitud,nombre estado,nhor,nombre municipio,Precipitación (litros/m2),Probabilidad de precipitación (%),raf,temp,Velocidad del viento (km/h)
0,Despejado,6,Este,107.1,15.4,Miércoles,20240918T18,81.9,20,54,17.3884,-97.229,Oaxaca,-1,Magdalena Zahuatlán,0.0,0.0,9.4,18.5,4.9
1,Cielo nublado,6,Este,91.8,14.6,Miércoles,20240918T19,87.5,20,54,17.3884,-97.229,Oaxaca,0,Magdalena Zahuatlán,0.8,0.0,6.5,16.7,3.9
2,Cielo nublado,6,Noreste,59.1,14.5,Miércoles,20240918T20,90.8,20,54,17.3884,-97.229,Oaxaca,1,Magdalena Zahuatlán,1.2,100.0,5.4,15.9,3.1
3,Cielo nublado,6,Noreste,47.5,14.2,Miércoles,20240918T21,91.2,20,54,17.3884,-97.229,Oaxaca,2,Magdalena Zahuatlán,1.4,100.0,4.8,15.7,3.0
4,Cielo nublado,6,Noreste,60.2,14.0,Miércoles,20240918T22,92.6,20,54,17.3884,-97.229,Oaxaca,3,Magdalena Zahuatlán,1.2,100.0,5.8,15.2,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285703,Poco nuboso,6,Este,80.1,20.7,Lunes,20240923T09,79.0,19,46,25.7553,-100.2896,Nuevo León,110,San Nicolás de los Garza,0.0,0.0,7.2,24.7,4.9
285704,Medio nublado,6,Este,95.6,20.5,Lunes,20240923T10,70.8,19,46,25.7553,-100.2896,Nuevo León,111,San Nicolás de los Garza,0.1,0.0,10.5,26.2,6.7
285705,Poco nuboso,6,Este,97.1,20.1,Lunes,20240923T11,62.8,19,46,25.7553,-100.2896,Nuevo León,112,San Nicolás de los Garza,0.1,0.0,12.2,27.8,8.9
285706,Poco nuboso,6,Este,100.7,19.7,Lunes,20240923T12,56.5,19,46,25.7553,-100.2896,Nuevo León,113,San Nicolás de los Garza,0.1,0.0,13.3,29.2,11.0


In [16]:
# Visualizando los nombres de los estados
#df['nombre estado'].unique()
sorted_unique_values = sorted(df['nombre estado'].unique())
sorted_unique_values

['Aguascalientes',
 'Baja California',
 'Baja California Sur',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Ciudad de México',
 'Coahuila',
 'Colima',
 'Durango',
 'Estado de México',
 'Guanajuato',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'Michoacán de Ocampo',
 'Morelos',
 'Nayarit',
 'Nuevo León',
 'Oaxaca',
 'Puebla',
 'Querétaro de Arteaga',
 'Quintana Roo',
 'San Luis Potosí',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaulipas',
 'Tlaxcala',
 'Veracruz de Ignacio de la Llave',
 'Yucatán',
 'Zacatecas']

In [17]:
df['nombre estado'].nunique()

32

In [18]:
df[df["nombre estado"] == "Coahuila"]["nombre municipio"].unique()

array(['Abasolo', 'Acuña', 'Allende', 'Arteaga', 'Candela', 'Castaños',
       'Cuatro Ciénegas', 'Escobedo', 'Francisco I. Madero', 'Frontera',
       'General Cepeda', 'Guerrero', 'Hidalgo', 'Jiménez', 'Juárez',
       'Lamadrid', 'Matamoros', 'Monclova', 'Morelos', 'Múzquiz',
       'Nadadores', 'Nava', 'Ocampo', 'Parras', 'Piedras Negras',
       'Progreso', 'Ramos Arizpe', 'Sabinas', 'Sacramento', 'Saltillo',
       'San Buenaventura', 'San Juan de Sabinas', 'San Pedro',
       'Sierra Mojada', 'Torreón', 'Viesca', 'Villa Unión', 'Zaragoza'],
      dtype=object)

In [19]:
df[(df['nombre estado'] == 'Coahuila') & (df['nombre municipio'] == 'Monclova')]


,Descripción del cielo,Diferencia respecto a hora UTC,Dirección del viento (Cardinal),Dirección del viento (Grados),dpt,dsem,hloc,hr,id estado,id municipio,Latitud,Longitud,nombre estado,nhor,nombre municipio,Precipitación (litros/m2),Probabilidad de precipitación (%),raf,temp,Velocidad del viento (km/h)
223532,Despejado,6,Noreste,34.2,19.0,Miércoles,20240918T18,46.8,5,18,26.9012,-101.4172,Coahuila,-1,Monclova,0.0,0.0,16.6,31.7,12.8
223533,Despejado,6,Noreste,27.4,20.0,Miércoles,20240918T19,55.9,5,18,26.9012,-101.4172,Coahuila,0,Monclova,0.0,0.0,28.5,29.7,12.2
223534,Despejado,6,Noreste,50.1,20.7,Miércoles,20240918T20,63.1,5,18,26.9012,-101.4172,Coahuila,1,Monclova,0.0,0.0,26.6,28.4,9.8
223535,Despejado,6,Este,93.7,21.2,Miércoles,20240918T21,69.0,5,18,26.9012,-101.4172,Coahuila,2,Monclova,0.0,0.0,14.1,27.5,5.2
223536,Despejado,6,Sureste,123.2,21.4,Miércoles,20240918T22,71.7,5,18,26.9012,-101.4172,Coahuila,3,Monclova,0.0,0.0,9.8,27.0,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223643,Despejado,6,Este,110.0,18.5,Lunes,20240923T09,61.0,5,18,26.9012,-101.4172,Coahuila,110,Monclova,0.1,0.0,5.1,26.7,5.3
223644,Despejado,6,Este,81.0,18.1,Lunes,20240923T10,55.1,5,18,26.9012,-101.4172,Coahuila,111,Monclova,0.1,0.0,10.2,27.9,7.4
223645,Poco nuboso,6,Noreste,64.7,17.8,Lunes,20240923T11,51.1,5,18,26.9012,-101.4172,Coahuila,112,Monclova,0.1,0.0,13.7,28.9,10.7
223646,Poco nuboso,6,Noreste,60.9,17.6,Lunes,20240923T12,48.5,5,18,26.9012,-101.4172,Coahuila,113,Monclova,0.0,0.0,14.8,29.5,12.6
